In [17]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [18]:
%cd drive/MyDrive/colab_projects/cocoapi/PythonAPI/

[Errno 2] No such file or directory: 'drive/MyDrive/colab_projects/cocoapi/PythonAPI/'
/Users/gbatch/Google Drive/colab_projects/cocoapi/PythonAPI


A lot of the code is from here:
* https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [2]:
import os

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import ToTensor

In [5]:
%cd ../utils/

from config import CATEGORIES_OF_INTEREST
from config import IMG_SIZE, NORMALIZATION_MEAN, NORMALIZATION_STD

from my_transforms import baseline_train_transforms, SimCLR_train_transforms, dev_transforms

from my_dataloaders import CocoNoCropping

%cd ../notebooks/

/Users/gbatch/Google Drive/colab_projects/cocoapi/utils
/Users/gbatch/Google Drive/colab_projects/cocoapi/notebooks


In [6]:
idx=1
with open('../my_splits/dev1_ids.txt', 'r') as f:
    img_ids = json.loads(f.read())
img_dir = '../my_images/dev1/'

    
img_id = str(img_ids[idx])
img_name = (12-len(img_id)) * '0' + img_id + '.jpg'

img_path = os.path.join(img_dir, img_name)
image = read_image(img_path)

print(image.shape)
print(image.shape[0])
print(image[0].shape)

if 1 == 1:
    image = torch.stack([image[0], image[0], image[0]], dim=0)
    
image.shape

torch.Size([3, 437, 583])
3
torch.Size([437, 583])


torch.Size([3, 437, 583])

In [7]:
%ls ../my_splits/

dev1_ids.txt    dev_ids.txt     test_ids.txt    train1_ids.txt  train_ids.txt


In [8]:
%ls ../my_annotations/

Icon?                               strCatNms_to_imgIds_train2017.json
imgIds_to_labels_train2017.json     strCatNms_to_imgIds_val2017.json
imgIds_to_labels_val2017.json


In [9]:
%ls ../images/

train2017/ val2017/


In [12]:
train1_dataset = CocoNoCropping(
    img_ids = '../my_splits/train1_ids.txt',
    my_annotations_file = '../my_annotations/imgIds_to_labels_train2017.json',
    img_dir = '../my_images/train1/',
    transform=baseline_train_transforms,
    divide_by_255=True
)

In [13]:
train1_dataloader = DataLoader(train1_dataset, batch_size=32, shuffle=True)

In [14]:
next(iter(train1_dataloader))

{'image': tensor([[[[-7.2049e-01, -6.1879e-01, -5.9153e-01,  ..., -2.3139e-01,
            -2.1339e-01, -2.1758e-01],
           [-6.6579e-01, -6.1844e-01, -5.5641e-01,  ..., -1.7232e-01,
            -1.5729e-01, -2.1583e-01],
           [-6.4727e-01, -6.1390e-01, -5.3736e-01,  ..., -1.4489e-01,
            -1.4314e-01, -2.1950e-01],
           ...,
           [-6.5863e-01, -5.7668e-01, -5.6357e-01,  ..., -2.2218e-02,
            -1.5928e-02, -3.2354e-02],
           [-6.5985e-01, -6.0463e-01, -6.3172e-01,  ..., -1.1116e-01,
            -9.4562e-02, -1.0120e-01],
           [-7.2346e-01, -6.4884e-01, -6.6719e-01,  ..., -7.1497e-02,
            -1.4611e-01, -1.8595e-01]],
 
          [[-7.7557e-01, -7.5342e-01, -6.6767e-01,  ..., -2.0731e-01,
            -2.0123e-01, -2.2552e-01],
           [-7.2894e-01, -7.3037e-01, -6.3194e-01,  ..., -1.4674e-01,
            -1.5371e-01, -2.3124e-01],
           [-7.2233e-01, -6.9071e-01, -6.5731e-01,  ..., -1.8462e-01,
            -1.9033e-01, -2.34

In [15]:
next(iter(train1_dataloader))['image'].min(), next(iter(train1_dataloader))['image'].max()

(tensor(-2.1179), tensor(2.6400))

In [16]:
for batch in train1_dataloader:
    inputs = batch['image'] # [batch size, channel, h, w]
    print(inputs.shape, inputs.dtype)
    
    # right now, `labels` is a list with num_classes tensors, each tensor with batch-size labels
    
    labels = torch.vstack(batch['label']).T.contiguous() 
    print(labels.shape, labels.dtype, labels.is_contiguous()) # labels
    print()
    #break

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 

In [21]:
dev1_dataset = CocoNoCropping(
    img_ids = '../my_splits/dev1_ids.txt',
    my_annotations_file = '../my_annotations/imgIds_to_labels_train2017.json',
    img_dir = '../my_images/dev1/',
    transform=dev_transforms,
    divide_by_255=True
)

In [24]:
dev1_dataloader = DataLoader(dev1_dataset, batch_size=32, shuffle=False)

In [25]:
for batch in dev1_dataloader:
    inputs = batch['image'] # [batch size, channel, h, w]
    print(inputs.shape, inputs.dtype)
    
    # right now, `labels` is a list with num_classes tensors, each tensor with batch-size labels
    
    labels = torch.vstack(batch['label']).T.contiguous() 
    print(labels.shape, labels.dtype, labels.is_contiguous()) # labels
    print()
    #break

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 3, 224, 224]) torch.float32
torch.Size([32, 4]) torch.int64 True

torch.Size([32, 